# <h1>Thêm thư viện cần thiết</h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

import plotly.express as px

import plotly.subplots as sp
import plotly.graph_objects as go

import numpy as np
import joblib
import torch
import random

from transformers import AutoModel, AutoTokenizer


## Các hàm cần thiết

In [40]:
def show_infor_dataframe(df):
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"Column '{column}' has {len(unique_values)} unique values:")
        print(unique_values)
        print("\n")

In [42]:
def display_column_info(df):
    for column in df.columns:
        # Kiểu dữ liệu của cột
        col_type = df[column].dtype
        
        # Số lượng giá trị không bị thiếu
        non_null_count = df[column].notna().sum()
        
        # Tổng số giá trị
        total_count = len(df[column])
        
        # Tính số lượng và tỷ lệ phần trăm giá trị thiếu
        missing_count = total_count - non_null_count
        missing_percentage = (missing_count / total_count) * 100
        
        
        print(f"Column: {column}")
        print(f"  Data Type: {col_type}")
        print(f"  Non-null Count: {non_null_count}")
        print(f"  Total Count: {total_count}")
        print(f"  Missing Values Count: {missing_count}")
        print(f"  Missing Values Percentage: {missing_percentage:.2f}%")
        print()
        # Kiểm tra xem tỷ lệ giá trị thiếu có cao không
        if missing_percentage >= 50:
            print(f"  Column {column} có {missing_percentage:.2f}% giá trị bị thiếu nhiều, cần xóa")
        print()
        print()

# <h1>Load dữ liệu file</h1>

In [22]:
df0 = pd.read_csv('data/diemthi_thpt_2020_2024/diem_thi_2020_2021.csv', header = 0)
df0.describe()

C:\Users\vongu\AppData\Local\Temp\ipykernel_19612\1844784900.py:1: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv('data/diemthi_thpt_2020_2024/diem_thi_2020_2021.csv', header = 0)


,SBD,Toán,Văn,Lý,Hoá,Sinh,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ,Year,code
count,1.857877e+06,1.843732e+06,1.830711e+06,638311.000000,642186.000000,631618.000000,1.203046e+06,1.183721e+06,1.015001e+06,1.640104e+06,1.857877e+06,1.857877e+06
mean,2.753798e+07,6.636630e+00,6.531775e+00,6.635257,6.664278,5.550259,5.071122e+00,6.870667e+00,8.257136e+00,5.247569e+00,2.020531e+03,2.752334e+01
std,1.911045e+07,1.745779e+00,1.312647e+00,1.444409,1.602272,1.401329,1.693421e+00,1.177897e+00,1.132359e+00,2.131752e+00,4.990093e-01,1.911970e+01
min,1.000001e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03,1.000000e+00
25%,1.000600e+07,5.400000e+00,5.750000e+00,5.750000,5.500000,4.500000,3.750000e+00,6.250000e+00,7.750000e+00,3.600000e+00,2.020000e+03,1.000000e+01
50%,2.801118e+07,7.000000e+00,6.750000e+00,7.000000,7.000000,5.500000,5.000000e+00,7.000000e+00,8.500000e+00,4.800000e+00,2.021000e+03,2.800000e+01
75%,4.400382e+07,8.000000e+00,7.500000e+00,7.750000,8.000000,6.500000,6.250000e+00,7.750000e+00,9.000000e+00,6.800000e+00,2.021000e+03,4.400000e+01
max,6.400684e+07,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,2.021000e+03,6.400000e+01


In [23]:
df1 = pd.read_csv('data/diemthi_thpt_2020_2024/diem_thi_thpt_2022.csv', header = 0)
df1.describe()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
count,9.954410e+05,982726.000000,981407.000000,870609.000000,325523.000000,327367.000000,322198.000000,659662.000000,657421.000000,554343.000000
mean,2.759692e+07,6.466254,6.507746,5.157686,6.723779,6.703898,5.019109,6.343930,6.677929,8.033266
std,1.917092e+07,1.675435,1.379602,1.949547,1.441925,1.600393,1.448270,1.612879,1.215013,1.067272
min,1.000001e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000217e+07,5.400000,5.580000,3.600000,5.750000,5.500000,4.000000,5.250000,5.750000,7.500000
50%,2.801886e+07,6.800000,6.500000,4.800000,7.000000,7.000000,4.750000,6.500000,6.750000,8.250000
75%,4.400582e+07,7.800000,7.500000,6.600000,7.750000,8.000000,6.000000,7.500000,7.500000,8.750000
max,6.400659e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [24]:
df2 = pd.read_csv('data/diemthi_thpt_2020_2024/diem_thi_thpt_2022.csv', header = 0)
df2.describe()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
count,9.954410e+05,982726.000000,981407.000000,870609.000000,325523.000000,327367.000000,322198.000000,659662.000000,657421.000000,554343.000000
mean,2.759692e+07,6.466254,6.507746,5.157686,6.723779,6.703898,5.019109,6.343930,6.677929,8.033266
std,1.917092e+07,1.675435,1.379602,1.949547,1.441925,1.600393,1.448270,1.612879,1.215013,1.067272
min,1.000001e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000217e+07,5.400000,5.580000,3.600000,5.750000,5.500000,4.000000,5.250000,5.750000,7.500000
50%,2.801886e+07,6.800000,6.500000,4.800000,7.000000,7.000000,4.750000,6.500000,6.750000,8.250000
75%,4.400582e+07,7.800000,7.500000,6.600000,7.750000,8.000000,6.000000,7.500000,7.500000,8.750000
max,6.400659e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [25]:
df3 = pd.read_csv('data/diemthi_thpt_2020_2024/diem_thi_thpt_2022.csv', header = 0)
df3.describe()

,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd
count,9.954410e+05,982726.000000,981407.000000,870609.000000,325523.000000,327367.000000,322198.000000,659662.000000,657421.000000,554343.000000
mean,2.759692e+07,6.466254,6.507746,5.157686,6.723779,6.703898,5.019109,6.343930,6.677929,8.033266
std,1.917092e+07,1.675435,1.379602,1.949547,1.441925,1.600393,1.448270,1.612879,1.215013,1.067272
min,1.000001e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000217e+07,5.400000,5.580000,3.600000,5.750000,5.500000,4.000000,5.250000,5.750000,7.500000
50%,2.801886e+07,6.800000,6.500000,4.800000,7.000000,7.000000,4.750000,6.500000,6.750000,8.250000
75%,4.400582e+07,7.800000,7.500000,6.600000,7.750000,8.000000,6.000000,7.500000,7.500000,8.750000
max,6.400659e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [26]:
mapping = {
    'SBD': 'sbd',
    'Toán': 'toan',
    'Văn': 'ngu_van',
    'Ngoại Ngữ': 'ngoai_ngu',
    'Lý': 'vat_li',
    'Hoá': 'hoa_hoc',
    'Sinh': 'sinh_hoc',
    'Lịch Sử': 'lich_su',
    'Địa Lý': 'dia_li',
    'GDCD': 'gdcd'
}

In [27]:
df0 = df0.rename(columns=mapping)
# Sau đó thực hiện nối như đã chỉ ở trên
df_combined = pd.concat([df0, df1, df2, df3], axis=0)
df_combined.columns

Index(['sbd', 'Tên', 'Ngày Sinh', 'Giới tính', 'toan', 'ngu_van', 'vat_li',
       'hoa_hoc', 'sinh_hoc', 'lich_su', 'dia_li', 'gdcd', 'ngoai_ngu', 'Year',
       'code', 'province'],
      dtype='object')

In [28]:
df_combined.describe()

,sbd,toan,ngu_van,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_li,gdcd,ngoai_ngu,Year,code
count,4.844200e+06,4.791910e+06,4.774932e+06,1.614880e+06,1.624287e+06,1.598212e+06,3.182032e+06,3.155984e+06,2.678030e+06,4.251931e+06,1.857877e+06,1.857877e+06
mean,2.757432e+07,6.531808e+00,6.516958e+00,6.688789e+00,6.688234e+00,5.229021e+00,5.862713e+00,6.750220e+00,8.118115e+00,5.192357e+00,2.020531e+03,2.752334e+01
std,1.914777e+07,1.704861e+00,1.354373e+00,1.443555e+00,1.601252e+00,1.453289e+00,1.755842e+00,1.204845e+00,1.097782e+00,2.022248e+00,4.990093e-01,1.911970e+01
min,1.000001e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03,1.000000e+00
25%,1.000366e+07,5.400000e+00,5.750000e+00,5.750000e+00,5.500000e+00,4.250000e+00,4.500000e+00,6.000000e+00,7.500000e+00,3.600000e+00,2.020000e+03,1.000000e+01
50%,2.801578e+07,6.800000e+00,6.500000e+00,7.000000e+00,7.000000e+00,5.000000e+00,6.000000e+00,6.750000e+00,8.250000e+00,4.800000e+00,2.021000e+03,2.800000e+01
75%,4.400503e+07,7.800000e+00,7.500000e+00,7.750000e+00,8.000000e+00,6.250000e+00,7.250000e+00,7.500000e+00,9.000000e+00,6.600000e+00,2.021000e+03,4.400000e+01
max,6.400684e+07,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,2.021000e+03,6.400000e+01


In [31]:
df_combined.isnull().sum()

sbd                0
Tên          4757833
Ngày Sinh    4757841
Giới tính    4757833
toan           52290
ngu_van        69268
vat_li       3229320
hoa_hoc      3219913
sinh_hoc     3245988
lich_su      1662168
dia_li       1688216
gdcd         2166170
ngoai_ngu     592269
Year         2986323
code         2986323
province     2986323
dtype: int64

In [32]:
# Xóa các cột không cần thiết cho dự đoán
df_combined = df_combined.drop(columns=['sbd', 'Tên','Ngày Sinh','Giới tính','Year','code','province'], errors='ignore')

In [36]:
# Điều kiện kiểm tra dòng có giá trị NaN trong các cột cụ thể
condition_nan_columns = df_combined[['toan', 'ngu_van', 'ngoai_ngu', 'vat_li', 'lich_su']].isna().all(axis=1)
# Xóa các dòng dữ liệu phù hợp
df_combined = df_combined[~condition_nan_columns]

In [38]:
show_infor_dataframe(df_combined)

Column 'toan' has 52 unique values:
[ 6.4  7.6  4.8  8.   8.2  5.4  7.4  5.   6.6  9.4  3.6  3.4  2.2  6.2
  7.8  5.8  5.6  9.   4.2  6.8  8.6  3.2  6.   7.   3.8  8.4  4.4  5.2
  4.   7.2  8.8  4.6  2.6  9.2  9.8  9.6  3.   1.6  2.   2.4  2.8  nan
  1.8 10.   1.4  1.2  0.6  1.   0.8  0.   0.2  0.4]


Column 'ngu_van' has 370 unique values:
[ 6.75  6.    4.75  7.    6.5   5.75  7.25  7.75  5.    3.5   4.5   5.5
  6.25  5.25  3.    7.5   4.25  8.    8.25   nan  8.5   4.    3.75  8.75
  3.25  9.    9.25  2.5   6.91  1.5   2.75  2.25  2.    6.4   9.5   1.75
  1.25  5.66  5.92  5.17  6.42  6.33  8.17  8.33  9.75  7.67  7.58  8.16
  9.08  7.83  7.42  6.67  0.    0.75  4.33  7.17  7.33  5.83  6.83  7.08
  3.92  4.92  6.08  5.33  0.25  4.67  6.16  6.41  6.17  6.85  6.55  6.44
  7.92  8.58  4.58  1.    7.8   6.8   7.1   7.4   8.1   8.42  4.42  5.08
  6.58  6.92  6.9   5.67  7.41  8.38  6.66  6.46  6.35  7.16  6.49  6.65
  4.91  5.58  7.91  7.13  5.7   6.2   5.23  4.73  4.3   4.66  6.3   3.33
 

In [43]:
display_column_info(df_combined)

Column: toan
  Data Type: float64
  Non-null Count: 4791910
  Total Count: 4844128
  Missing Values Count: 52218
  Missing Values Percentage: 1.08%



Column: ngu_van
  Data Type: float64
  Non-null Count: 4774932
  Total Count: 4844128
  Missing Values Count: 69196
  Missing Values Percentage: 1.43%



Column: vat_li
  Data Type: float64
  Non-null Count: 1614880
  Total Count: 4844128
  Missing Values Count: 3229248
  Missing Values Percentage: 66.66%

  Column vat_li có 66.66% giá trị bị thiếu nhiều, cần xóa


Column: hoa_hoc
  Data Type: float64
  Non-null Count: 1624270
  Total Count: 4844128
  Missing Values Count: 3219858
  Missing Values Percentage: 66.47%

  Column hoa_hoc có 66.47% giá trị bị thiếu nhiều, cần xóa


Column: sinh_hoc
  Data Type: float64
  Non-null Count: 1598189
  Total Count: 4844128
  Missing Values Count: 3245939
  Missing Values Percentage: 67.01%

  Column sinh_hoc có 67.01% giá trị bị thiếu nhiều, cần xóa


Column: lich_su
  Data Type: float64
  Non-null

Tính toán theo dòng dữ liệu xem có những tổ hợp mà dòng dữ liệu ấy đạt được. <br>
Chọn ra tổ hợp môn có điểm tổng cao nhất sau đó lưu điểm của từng môn trong tổ hợp có điểm cao nhất <br>thành điểm môn thứ nhất, môn thứ 2, môn thứ 3 (ScoTHPT1, ScoTHPT2, ScoTHPT3)

In [44]:
to_hop_mon = {
    'A00': ['toan', 'vat_li', 'hoa_hoc'],
    'A01': ['toan', 'vat_li', 'ngoai_ngu'],
    'B00': ['toan', 'hoa_hoc', 'sinh_hoc'],
    'C00': ['ngu_van', 'lich_su', 'dia_li'],
    'D01': ['ngu_van', 'toan', 'ngoai_ngu'],
    'D14': ['ngu_van', 'lich_su', 'ngoai_ngu'],
    'D15': ['ngu_van', 'ngoai_ngu', 'dia_li']
}


In [69]:
def find_best_combination(row, to_hop_mon):
    # Biến lưu tổng điểm cao nhất và tổ hợp môn tương ứng
    max_total_score = -1
    best_combination = None
    best_subjects_scores = [None, None, None]

    # Duyệt qua từng tổ hợp môn
    for combo, subjects in to_hop_mon.items():
        # Lấy điểm các môn trong tổ hợp và bỏ qua nếu có giá trị NaN
        scores = [row[subject] for subject in subjects if pd.notna(row[subject])]
        
        # Nếu không có đủ 3 môn thi, bỏ qua tổ hợp này
        if len(scores) < 3:
            continue

        # Tính tổng điểm của tổ hợp này
        total_score = sum(scores)

        # So sánh với tổng điểm cao nhất hiện tại
        if total_score > max_total_score:
            max_total_score = total_score
            best_combination = combo  # Lưu lại tên tổ hợp môn
            best_subjects_scores = sorted(scores, reverse=True)  # Sắp xếp điểm theo thứ tự giảm dần

    # Trả về tên tổ hợp môn và điểm của các môn trong tổ hợp
    return pd.Series([best_combination] + best_subjects_scores[:3])

df = df_combined
# Áp dụng hàm cho từng dòng dữ liệu
df[['To_hop', 'ScoTHPT1', 'ScoTHPT2', 'ScoTHPT3']] = df.apply(find_best_combination, axis=1, to_hop_mon=to_hop_mon)

In [70]:
df.isnull().sum()

toan           52218
ngu_van        69196
vat_li       3229248
hoa_hoc      3219858
sinh_hoc     3245939
lich_su      1662096
dia_li       1688166
gdcd         2166107
ngoai_ngu     592197
ScoTHPT1       31222
ScoTHPT2       31222
ScoTHPT3       31222
To_hop         31222
dtype: int64

In [71]:
# Điều kiện kiểm tra dòng có giá trị NaN trong các cột cụ thể
condition_nan_columns = df[['To_hop','ScoTHPT1', 'ScoTHPT2', 'ScoTHPT3']].isna().all(axis=1)
# Xóa các dòng dữ liệu phù hợp
df = df[~condition_nan_columns]

In [72]:
df.isnull().sum()

toan           49390
ngu_van        65326
vat_li       3198466
hoa_hoc      3189353
sinh_hoc     3214927
lich_su      1656320
dia_li       1658449
gdcd         2135957
ngoai_ngu     563841
ScoTHPT1           0
ScoTHPT2           0
ScoTHPT3           0
To_hop             0
dtype: int64

# <h1>Chuyển hóa số liệu</h1>

**ta chuyển hóa dữ liệu về dạng số để thực hiện trực quan hóa số liệu:**
- `ScoTHPT1`,`ScoTHPT2`,`ScoTHPT3` : (numeric 1: 8.5 - 10, 2: 7.0 - 8.4, 3: 5.5 - 6.9, 4: 4.0 - 5.4, 5: <= 4.0)
- `To_hop` : load lại file label encoder trước đó
- `TB_HK1`: (numeric 0: điểm trên khá(2.5); 1: điểm dưới khá(2.49) )

In [74]:
# Chuyển hóa cột 'ScoTHPT1', 'ScoTHPT2', 'ScoTHPT3'
def score_to_ScoTHPTnumeric(diem):
    if 8.5 <= diem <= 10:
        return 0
    elif 7.0 <= diem < 8.5:
        return 1
    elif 5.5 <= diem < 7.0:
        return 2
    elif 4.0 <= diem < 5.5:
        return 3
    elif 0 <= diem < 4.0:
        return 4
    else:
        return 999

In [75]:
for col in ['ScoTHPT1', 'ScoTHPT2', 'ScoTHPT3']:
    df[col] = df[col].apply(score_to_ScoTHPTnumeric)

C:\Users\vongu\AppData\Local\Temp\ipykernel_19612\3321737424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(score_to_ScoTHPTnumeric)


In [76]:
label_encoder = joblib.load('label_encoder/To_hop_labelencoder.pkl')
df['To_hop'] = label_encoder.transform(df['To_hop'])

C:\Users\vongu\AppData\Local\Temp\ipykernel_19612\3228978216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['To_hop'] = label_encoder.transform(df['To_hop'])


In [77]:
show_infor_dataframe(df)

Column 'toan' has 52 unique values:
[ 6.4  7.6  4.8  8.   8.2  5.4  7.4  5.   6.6  9.4  3.6  3.4  2.2  6.2
  7.8  5.8  5.6  9.   4.2  6.8  8.6  3.2  6.   7.   3.8  8.4  4.4  5.2
  4.   7.2  8.8  4.6  2.6  9.2  9.8  9.6  3.   1.6  2.   2.4  2.8  nan
  1.8 10.   1.4  1.2  0.6  1.   0.8  0.2  0.   0.4]


Column 'ngu_van' has 367 unique values:
[ 6.75  6.    4.75  7.    6.5   5.75  7.25  7.75  5.    3.5   4.5   5.5
  6.25  5.25  3.    7.5   4.25  8.    8.25   nan  8.5   4.    3.75  8.75
  3.25  9.    9.25  2.5   6.91  1.5   2.75  2.25  6.4   2.    9.5   1.75
  1.25  5.66  5.92  5.17  6.42  6.33  8.17  8.33  9.75  7.67  7.58  8.16
  9.08  7.83  7.42  6.67  0.    0.75  4.33  7.17  7.33  5.83  6.83  7.08
  6.08  5.33  4.67  6.16  6.41  6.17  6.85  6.55  6.44  7.92  8.58  4.58
  1.    7.8   6.8   7.1   7.4   8.1   8.42  4.42  5.08  6.58  6.92  6.9
  5.67  7.41  8.38  6.66  6.46  6.35  7.16  4.92  6.49  6.65  4.91  5.58
  7.91  7.13  5.7   6.2   5.23  4.73  4.3   4.66  6.3   3.33  7.6   6.1
  5

In [78]:
df.to_csv('data/data_dudoan_sco123.csv')

# Test dữ liệu với các mô hình đã đào tạo được

In [79]:
df_final = pd.read_csv('data/data_dudoan_sco123.csv')
df_final = df_final.drop(columns=['Unnamed: 0'])

In [80]:
X = df_final[['ScoTHPT1','ScoTHPT2', 'ScoTHPT3','To_hop']]


In [81]:
RFC_Model = 'model/RFC_Model_ScoTHPT123.pkl'

In [82]:
# Tải các mô hình đã huấn luyện từ file
RFC_Model = joblib.load(RFC_Model)

In [83]:
df_final['TB_HK1'] = RFC_Model.predict(X)

In [86]:
def numeric_to_score(diem_numeric):
    if diem_numeric == 0:
        return (8.5, 10)
    elif diem_numeric == 1:
        return (7.0, 8.5)
    elif diem_numeric == 2:
        return (5.5, 7.0)
    elif diem_numeric == 3:
        return (4.0, 5.5)
    elif diem_numeric == 4:
        return (0, 4.0)
    else:
        return 'Invalid'


In [87]:
# Áp dụng hàm mã hóa ngược cho các cột điểm
for col in ['ScoTHPT1', 'ScoTHPT2', 'ScoTHPT3']:
    df_final[col + '_raw'] = df_final[col].apply(numeric_to_score)

In [88]:
# Tải LabelEncoder đã lưu
label_encoder = joblib.load('label_encoder/To_hop_labelencoder.pkl')

In [89]:
# Hàm mã hóa ngược cho cột To_hop
def decode_label(encoded_label, label_encoder):
    return label_encoder.inverse_transform([encoded_label])[0]

In [90]:
# Áp dụng hàm mã hóa ngược cho cột 'To_hop'
df_final['To_hop_raw'] = df_final['To_hop'].apply(lambda x: decode_label(x, label_encoder))

In [93]:
def numeric_result(kq):
    if kq == 0:
        return 'Trên Khá'
    elif kq == 1:
        return 'Dưới Khá'
df_final['TB_HK1_raw'] = df_final['TB_HK1'].apply(numeric_result)

In [94]:
df_final.to_csv('data/KetQua_sco123.csv')

In [84]:
df_zero = df_final[df_final['TB_HK1'] == 0]

# In các dòng dữ liệu này
print(df_zero)

         toan  ngu_van  vat_li  hoa_hoc  sinh_hoc  lich_su  dia_li  gdcd  \
4         8.2     6.50    8.00     8.50      5.00      NaN     NaN   NaN   
9         9.4     7.00    7.50     7.75      4.50      NaN     NaN   NaN   
11        8.0     6.75    7.25     7.75      4.75      NaN     NaN   NaN   
15        8.2     6.75    8.50     8.25      5.00      NaN     NaN   NaN   
26        9.0     6.50    8.00     8.25      6.00      NaN     NaN   NaN   
...       ...      ...     ...      ...       ...      ...     ...   ...   
4812754   8.8     6.50    7.00     3.50      5.00      NaN     NaN   NaN   
4812779   8.4     8.50     NaN      NaN       NaN      9.0    9.25  8.75   
4812844   7.8     6.75    8.00     8.75      8.50      NaN     NaN   NaN   
4812860   8.2     5.50    7.50     8.75      5.75      NaN     NaN   NaN   
4812865   7.0     4.75    8.00     7.50      2.50      NaN     NaN   NaN   

         ngoai_ngu  ScoTHPT1  ScoTHPT2  ScoTHPT3  To_hop  TB_HK1  
4              4.0  